## Imports

In [1]:
import os
from urllib.request import urlopen
import gzip
import shutil
from itertools import islice
import xml.etree.cElementTree as ET
import datetime
import requests
import json
import time
from elasticsearch import Elasticsearch
from elasticsearch import helpers

## Download file if not exist

In [2]:
file_url = 'https://dumps.wikimedia.org/plwiki/latest/plwiki-latest-abstract.xml.gz'
file_name = "plwiki-latest-abstract.xml"

start_time = time.time()

archive_name = file_url.split('/')[-1]

outfile = open(archive_name, "wb")
outfile.write(urlopen(file_url).read())
outfile.close()
with gzip.open(archive_name, 'rb') as f_in:
    with open(file_name, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
elapsed_time = time.time() - start_time
print("download took: " + str(round(elapsed_time)) + " seconds")

download took: 32 seconds


## Show file

In [3]:
N = 50

with open(file_name, encoding="utf8") as myfile:
    head = list(islice(myfile, N))

for row in list(head):
    print(row)

<feed>

<doc>

<title>Wikipedia: AWK</title>

<url>https://pl.wikipedia.org/wiki/AWK</url>

<abstract>AWK – interpretowany język programowania, którego główną funkcją jest wyszukiwanie i przetwarzanie wzorców w plikach lub strumieniach danych. Jest także nazwą programu początkowo dostępnego dla systemów operacyjnych będących pochodnymi UNIX-a, obecnie także na inne platformy.</abstract>

<links>

<sublink linktype="nav"><anchor>Struktura programów AWK</anchor><link>https://pl.wikipedia.org/wiki/AWK#Struktura_programów_AWK</link></sublink>

<sublink linktype="nav"><anchor>Komendy i składnia</anchor><link>https://pl.wikipedia.org/wiki/AWK#Komendy_i_składnia</link></sublink>

<sublink linktype="nav"><anchor>print</anchor><link>https://pl.wikipedia.org/wiki/AWK#print</link></sublink>

<sublink linktype="nav"><anchor>Zmienne i tablice</anchor><link>https://pl.wikipedia.org/wiki/AWK#Zmienne_i_tablice</link></sublink>

<sublink linktype="nav"><anchor>Zmienne</anchor><link>https://pl.wikipedia

## Parse file to XML structure and show no of index

In [4]:
root = ET.parse(file_name).getroot()
len(root)

1383604

## Some articles have not been well exported to the xml file, so I should skip them. I use a simple scheme - if the description is not long enough, I ignore it. It will also be useful for further exploration of documents, because too short entries are worthless

In [5]:
datas = []

for doc in list(root.findall('doc')):
    title = doc.find('title').text
    content = doc.find('abstract').text
    if (content == None):
        continue
    if (len(content.replace(" ", "")) > 40):
        data = {}
        try:
          data['Title'] = title.replace("Wikipedia: ", "")
        except:
          continue
        data['Content'] = content
        datas.append(data)

## Preview of the prepared structure that will be added to the elasticsearch database

In [6]:
for data in datas[:10]:
    print(data['Title'])
    print(data['Content'])

AWK
AWK – interpretowany język programowania, którego główną funkcją jest wyszukiwanie i przetwarzanie wzorców w plikach lub strumieniach danych. Jest także nazwą programu początkowo dostępnego dla systemów operacyjnych będących pochodnymi UNIX-a, obecnie także na inne platformy.
Alergologia
Alergologia – dziedzina medycyny zajmująca się rozpoznawaniem i leczeniem schorzeń alergicznych.
ASCII
ASCII (czyt. aski, skrót od ang. American Standard Code for Information Interchange) – siedmiobitowy system kodowania znaków, używany we współczesnych komputerach oraz sieciach komputerowych, a także innych urządzeniach wyposażonych w mikroprocesor. Przyporządkowuje liczbom z zakresu 0−127: litery alfabetu łacińskiego języka angielskiego, cyfry, znaki przestankowe i inne symbole oraz polecenia sterujące. Na przykład litera „a” jest kodowana jako liczba 97, a znak spacji jest kodowany jako 32. Większość współczesnych systemów kodowania znaków jest rozszerzeniem standardu ASCII.
Atom
Atom – podstawo

## Loaded XML file should be deleted from memory, because it requires a lot of memory

In [7]:
root.clear()
del root

## Number of elements after filtering based on length. It is important how much has been ignored

In [8]:
len(datas)

801499

## Elastic init

In [9]:
es_host = '192.168.1.107'
es_port = 9200
es_index_name = 'wiki_abstract_pl'

es = Elasticsearch(hosts = [{'host': es_host, 'port': es_port}])

## Remove old index

In [10]:
try:
    es.indices.delete(index = es_index_name)
except:
    print("Index not created or already deleted")

## We can now add entries to the database using the configured elasticsearch database API

In [11]:
start_time = time.time()

actions = [
    {
        "_index": es_index_name,
        "_source": {
            "title": data['Title'],
            "content": data['Content'],
            "timestamp": datetime.datetime.now()}
    }
    for data in datas
]

helpers.bulk(es, actions)

print("done")

elapsed_time = time.time() - start_time
print("adding documents took: " + str(round(elapsed_time)) + " seconds")

done
adding documents took: 124 seconds


## Check if files exists (response = 200)

In [12]:
index_url = "http://192.168.1.107:9200/wiki_abstract_pl/"
response = requests.get(index_url)
print("Response code: " + str(response.status_code))

Response code: 200
